In [1]:
import pandas as pd

In [2]:
# Load the feather file into a pandas dataframe
df_data = pd.read_feather("results_v3.feather")

In [3]:
print(df_data.iloc[0])

index                                                           0
year                                                         2022
title           US military creates space unit in South Korea ...
url             https://www.airforcetimes.com/battlefield-tech...
text_content    SEOUL, South Korea — The U.S. military formall...
Name: 0, dtype: object


---
The *News Labels Classifier* is running on the local machine.  
Text contents are sended to it.  
In response, the *News Labels Classifier* sends the scriptors and their score.

In [4]:
import requests
import json

In [5]:
headers = {
    "Content-Type": "application/json"
}
data = {
    "text" : df_data.iloc[0].text_content # first record of the dataframe
}
response = requests.post("http://localhost:8000/predict.json", data=json.dumps(data), headers=headers)
print(response.status_code)

200


In [6]:
response_text = json.loads(response.text)
# Retrieve the descriptors ( predicted tags with e score)
descriptors = (response_text['predictions']['allDescriptors'])
print(descriptors[0:3]) # Extract the 3 first  

[{'label': 'armament, defense and military forces', 'score': '0.70348'}, {'label': 'missiles and missile defense systems', 'score': '0.38117'}, {'label': 'international relations', 'score': '0.26588'}]
